<a href="https://colab.research.google.com/github/MahdiFaourr/MahdiFaourr/blob/main/chest_ctscan_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install useful libraries
!pip install opendatasets
!pip install mlflow
!pip install tensorflow-model-optimization

In [ ]:
# Import necessary libraries and functions
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
import opendatasets as od
import mlflow
import mlflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_model_optimization.sparsity import keras as sparsity

In [ ]:
# Dowload the data to your working directory
od.download("https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images")

In [4]:
# Create an ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1)


In [ ]:
# Create train,valid and test generators
train_generator=datagen.flow_from_directory("/content/chest-ctscan-images/Data/train",class_mode="categorical",batch_size=36,shuffle=True)
valid_generator=datagen.flow_from_directory("/content/chest-ctscan-images/Data/valid",class_mode="categorical",batch_size=36,shuffle=True)
test_generator=datagen.flow_from_directory("/content/chest-ctscan-images/Data/test",class_mode="categorical",batch_size=36,shuffle=True)

In [ ]:
# Create and set an experiment
exp=mlflow.create_experiment("first_experiment")
mlflow.set_experiment("first_experiment")

In [8]:
# Create a model
def get_model(params):
    # Load MobileNet as a feature extractor
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    # Create a Sequential model for building custom layers
    model = Sequential()

    # Add the MobileNet feature extractor to the model
    model.add(base_model)

    # Add global average pooling and custom dense layers
    model.add(GlobalAveragePooling2D())
    model.add(Dense(params['neurons'], activation='relu'))
    model.add(Dense(4, activation='softmax'))
    return model


In [9]:
# Train the created model
def train_model(params,text_params,train_generator,valid_generator):
  model=get_model(params)
  model.compile(optimizer=text_params['optimizer'],loss='categorical_crossentropy',metrics=['accuracy'])
  history=model.fit(train_generator,validation_data=valid_generator,epochs=params['epochs'],batch_size=params['batch_size'])
  return history, model

In [10]:
# Create a function to log the parameters
def mlflow_log(params, history, text_params, model, run_num):
    # Taking the last value for each metric in history
    metrics = history.history

    for key in metrics.keys():
        metrics[key] = metrics[key][-1]

    # Begin to log for the set experiment
    with mlflow.start_run() as run:

        # log params
        mlflow.log_params(params)

        # log metrics
        mlflow.log_metrics(metrics)

        # log text params
        for text_item in text_params.items():
            if isinstance(text_item[1], list):
                for metric_name in text_item[1]:
                    mlflow.log_text(metric_name, text_item[0] + '.txt')
            else:
                mlflow.log_text(text_item[1],text_item[0] + '.txt')

        # log Keras model
        mlflow.keras.log_model(model, artifact_path=f"model_{run_num}")

In [ ]:
# 1st run
params = {
    'neurons':85,
    'epochs':8,
    'batch_size':32
}

text_params = {
    'optimizer':'adam'
}


run_num =1

history, model = train_model(params, text_params,train_generator, valid_generator)

print("[INFO] Logging the parameters, textual info & best values for each metric & the model...")
mlflow_log(params, history, text_params, model, run_num)

In [ ]:
# 2nd run
params = {
    'neurons':90,
    'epochs':15,
    'batch_size':32
}

text_params = {
    'optimizer':'adam'
}


run_num =2

history, model = train_model(params, text_params,train_generator, valid_generator)

print("[INFO] Logging the parameters, textual info & best values for each metric & the model...")
mlflow_log(params, history, text_params, model, run_num)

In [ ]:
# 3rd run
params = {
    'neurons':85,
    'epochs':12,
    'batch_size':32
}

text_params = {
    'optimizer':'sgd'
}


run_num =3

history, model = train_model(params, text_params,train_generator, valid_generator)

print("[INFO] Logging the parameters, textual info & best values for each metric & the model...")
mlflow_log(params, history, text_params, model, run_num)

In [ ]:
# 4th run
params = {
    'neurons':85,
    'epochs':12,
    'batch_size':25
}

text_params = {
    'optimizer':'sgd'
}


run_num =4

history, model = train_model(params, text_params,train_generator, valid_generator)

print("[INFO] Logging the parameters, textual info & best values for each metric & the model...")
mlflow_log(params, history, text_params, model, run_num)

In [ ]:
# Search through runs
mlflow.search_runs("first_experiment")

In [19]:
# Create an EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

In [20]:
# Create and compile the final model
def final_model():
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(85, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    model.compile(optimizer="sgd",loss="categorical_crossentropy",metrics=['accuracy'])
    return model

In [ ]:
# Train your model with callback
model=final_model()
model.fit(train_generator, validation_data=valid_generator,epochs=12,batch_size=32,callbacks=[early_stopping],verbose=1)

In [ ]:
# Evaluate the model on test_generator
model.evaluate(test_generator)

In [ ]:
# Save the model to a file
model.save("my_model.h5")